RUN THIS CODE TO ANALYZE THE STRIA AND STRIA VASCULATURE LABELS 

In [ ]:
!pip install feret

In [ ]:
import numpy as np
import os
from skimage.io import imsave, imread
import pandas as pd
import util_stria

In [ ]:
def stria_analyze(imagesloc,strialoc,caploc):
    images=[]
    striamasks=[]
    vasmasks=[]
    analysisdata=[]
    cellmask=[]
    
    dir_list = os.listdir(imagesloc)
    for name in dir_list:
        if((name[-4:] != '.png') | (name=='.tiff.png')|(name[0:2]=='._')):
            continue
        try:
            maskname=name[:-9]+".png"
            im=imread(imagesloc+"/"+name)
            maskstria=imread(strialoc+"/"+maskname)
            maskvas=imread(caploc+"/"+maskname)
            print(name)
        except: 
            print("image not found")
            continue
        maskstria=maskstria>0
        maskvas=maskvas>0
        maskstria=util_stria.returnlargarea(maskstria,strialoc+"/"+maskname)
        
        striaarea,totalcaparea,avgareapercap,averageecc=util_stria.areasandecc(maskstria,maskvas)
        labelim,gray,segmented_image= util_stria.kmeanstria(im,maskstria)
        cellval,capval=util_stria.calcmaxminlabel(gray,labelim,4)
        cell=(labelim==cellval)

        numcell=util_stria.countcells(cell)
        w1,w2,w3=util_stria.width(maskstria)
        images.append(im)
        striamasks.append(maskstria)
        vasmasks.append(maskvas)
        cellmask.append(cell)
        analysisdata.append([name,striaarea,totalcaparea,avgareapercap,averageecc,numcell,w1,w2,w3])
    df = pd.DataFrame(analysisdata)
    df.columns=['Image','Stria Area(px)','Total capillary area(px)','Average capillary area','Average capillary eccentricity','number of nuclei','average width of stria','feret width of stria','width at midline of stria']
    df.to_csv('analysistotal.csv')
    return df

In [ ]:
### SET THESE VARIABLES BEFORE RUNNING SCRIPT
imagesloc=".../stria/" ##Location of RGB Stria patches 
strialoc=".../striaanalysis/" ##Location of binary masks outlining stria 
vasloc=".../vasanalysis/" ##Location of binary vasculature masks 

df=stria_analyze(imagesloc,strialoc,vasloc) 

In [ ]:
df

In [ ]:
df